# Paillier Crypto

## Generate Public and Private Key

In [1]:
from openfed.hooks.cypher.paillier_crypto import key_gen
import os
import torch

if not os.path.isfile('/tmp/public.key') or not os.path.isfile('/tmp/private.key'):
    public_key, private_key = key_gen()
    torch.save(public_key, '/tmp/public.key')
    torch.save(private_key, '/tmp/private.key')
    print("Save new key to /tmp/public.key and /tmp/private.key")
else:
    private_key = torch.load('/tmp/private.key')
    print("Load private key from /tmp/private.key")
    print(private_key)

Load private key from /tmp/private.key
<OpenFed> PrivateKey
n_lwe: 3000, bits: 32, l: 64, bound: 8, p: 4294967297, q: 4294967296



/Users/densechen/miniconda3/envs/openfed/lib/python3.7/site-packages/openfed-0.0.0-py3.7.egg/openfed/third_party/mmcv/__init__.py:13: UserWarning: `openfed.third_party.mmcv` module requires `mmcv` package, but not found. You can install it from https://github.com/open-mmlab/mmcv.


## Network

In [2]:
import torch.nn as nn

network = nn.Linear(784, 10)
loss_fn = nn.CrossEntropyLoss()

## Aggregator

In [3]:
from openfed.optim import PaillierOp, build_aggregator

agg_op = PaillierOp(network.parameters(), private_key)

aggregator = build_aggregator(agg_op)

print(aggregator)

<OpenFed> Gluer
Gluer_PaillierOp_ReduceOp



## Build Optimizer

In [4]:
import torch

from openfed.optim import build_fed_optim

optim = torch.optim.SGD(network.parameters(), lr=1.0)
fed_optim = build_fed_optim(optim)

print(fed_optim)

<OpenFed> Gluer
Gluer_SGD_Penalizer



## World

In [5]:
from openfed.core import World, leader

world = World(role=leader, dal=False, mtt=5)

print(world)

<OpenFed> World
+----------------+-------+-------+-----+------+
|      role      | alive |  dal  | mtt | pipe |
+----------------+-------+-------+-----+------+
| openfed_leader |  True | False |  5  |  0   |
+----------------+-------+-------+-----+------+



## API

In [6]:
from openfed import API
openfed_api = API(
    world=world,
    state_dict=network.state_dict(keep_vars=True),
    fed_optim=fed_optim,
    aggregator=aggregator)

print(openfed_api)

<OpenFed> OpenFedAPI
ROLE: openfed_leader



# Step

In [7]:
from openfed.hooks import Aggregate

with openfed_api: 
    aggregate = Aggregate(
        activated_parts=dict(train=2),
        max_version=5,
    )
    print(aggregate)

<Round: 1> train:   0%|          | 0/2 [00:00<?, ?it/s]

<OpenFed> AggregateStep
period: 1 day, 0:00:00, activated_parts: {'train': 2}, checkpoint: None
max_loop_tiems: -1, max_version: 5



## Address

In [8]:
from openfed.common import default_tcp_address

address = default_tcp_address

print(address)

Address(backend='gloo', init_method='tcp://localhost:1994', world_size=2, rank=-1, store=None, group_name='OpenFed')


## Connection

In [9]:
import time
openfed_api.build_connection(address=address)

print(openfed_api.maintainer)

<OpenFed> Maintainer
+---------+----------+---------+
| Pending | Finished | Discard |
+---------+----------+---------+
|    1    |    0     |    0    |
+---------+----------+---------+



## Loop

In [10]:
openfed_api.run()

<Round: 5> train: 100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


## Finished

In [11]:
openfed_api.finish()

print("Finished")

Finished
